In [178]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import json
import io
import time
import re
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [180]:
def train_test_split(n = 0.7):
    time1 = time.time()
    # import files
    review_json_file = '/Users/mjy/Downloads/yelp_dataset/review.json'

    review = []
    for line in open(review_json_file, 'r'):
        review.append(json.loads(line))

    # convert to dataframe
    review_df = pd.DataFrame.from_records(review)
    # extract the userful column
    review_df = review_df.loc[:,['business_id','user_id','stars','text']]
    # split the test and training dataset
    length = int(len(review_df) * n)

    review_df_training = review_df.iloc[:length,]
    review_df_test = review_df.iloc[length:,]
    review_df_training.to_csv('training_large.csv')
    #review_df_test.to_csv('test.csv')
    time2 = time.time()
    print('SUCCESS!!!  train_test_split')
    print('The training set has ', length, 'rows data')
    print('The testing set has ', len(review_df) - length, 'rows data')
    print('Time: ', time2 - time1)

In [1]:
def textProcessing(text):
    # lower words  
    text = text.lower()
    # remove punctuation
    for c in string.punctuation:
        text = text.replace(c, ' ')
    # tokenize
    wordLst = nltk.word_tokenize(text)
    # stop word
    filtered = [w for w in wordLst if w not in stopwords.words('english')]
    # keep noun  
    refiltered =nltk.pos_tag(filtered)
    filtered = [w for w, pos in refiltered if pos.startswith('NN')]
    # xtract the stem
    ps = PorterStemmer()
    filtered = [ps.stem(w) for w in filtered]

    return " ".join(filtered) 

def rating_proportion(text,rate):
    return text * int(rate)

In [2]:
def lda(review,n_topic = 10,n_top_words=20):
# vectorization
# generate the word-docu matrix
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,stop_words='english')
    tf = tf_vectorizer.fit_transform(review)
# train the lda model
    lda = LatentDirichletAllocation(n_topics=n_topic, 
                                max_iter=50,
                                learning_method='batch')
    lda.fit(tf)
# print the performance
    print('perplexity is: ',lda.perplexity(tf))

# generate the top word list for every topic
    tf_feature_names = tf_vectorizer.get_feature_names()
    feature_dict = {k: v for v, k in enumerate(tf_feature_names)}

    for topic_idx, topic in enumerate(lda.components_):       
        print ("Topic #%d:" % topic_idx)
        print (" ".join([tf_feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))    

# return the topic*word distribution matrix
    return lda.components_,feature_dict

In [ ]:
def topic_probability(document,feature_dict,topic_word_matrix):
    word_list = document.split()
    topic_num = len(topic_word_matrix)
    topic_probability = {k:0 for k in range(topic_num)}
    for topic_idx in range(topic_num):
        for word in word_list:
            if word in feature_dict.keys():
                topic_probability[topic_idx] += topic_word_matrix[topic_idx,feature_dict[word]]
    return topic_probability

In [199]:
import math
def normalize(x):
    normalized_list = []
    new_dict = {}
    for key,value in x.items():
        normalized_list.append(value**2)
    for key,value in x.items():
        if sum(normalized_list) == 0:
            new_dict[key] = 0
        else:
            new_dict[key] = value/math.sqrt(sum(normalized_list))
    return new_dict

In [181]:
train_test_split(n = 0.008)

SUCCESS!!!  train_test_split
The training set has  53487 rows data
The testing set has  6632413 rows data
Time:  604.2358219623566


In [183]:
training = pd.read_csv('training_large.csv')

In [186]:
training.head()

,Unnamed: 0,business_id,user_id,stars,text
0,0,ujmEBvifdJM6h6RLv4wQIg,hG7b0MtEbXx5QzbzE6C_VA,1.0,Total bill for this horrible service? Over $8G...
1,1,NZnhc2sEQy3RmzKTZnqtwQ,yXQM5uF2jS6es16SJzNHfg,5.0,I *adore* Travis at the Hard Rock's new Kelly ...
2,2,WTqjgwHlXbSFevF32_DJVw,n6-Gk65cPZL6Uz8qRm3NYw,5.0,I have to say that this office really has it t...
3,3,ikCg8xy5JIg_NGPx-MSIDA,dacAIZ6fTM6mqwW5uxkskg,5.0,Went in for a lunch. Steak sandwich was delici...
4,4,b1b1eb3uo-w561D0ZfCEiQ,ssoyf2_x0EQMed6fgHeMyQ,1.0,Today was my second out of three sessions I ha...


In [190]:
#generate user_id, biz_id dict
user_raw_int_id_dic = {v: k for k,v in enumerate(training['user_id'].unique())}
biz_raw_int_id_dic = {v: k for k,v in enumerate(training['business_id'].unique())}

#transform str_id to int_id and keep them in the dataframe
training['user_int_id'] = training['user_id'].apply(lambda x: user_raw_int_id_dic[x])
training['biz_int_id'] = training['business_id'].apply(lambda x: biz_raw_int_id_dic[x])

#Generate Rating Sparse Matrix
user_row = training['user_int_id'].values
biz_column = training['biz_int_id'].values

user_num = len(user_raw_int_id_dic)
biz_num = len(biz_raw_int_id_dic)

rating_data = training['stars'].values
rating_matrix = sparse.csr_matrix((rating_data, (user_row, biz_column)), shape=(user_num, biz_num))
#c.todense()
#c.toarray()

training['text'] = training['text'].apply(textProcessing)
training['text'] = training['text'] * training['stars'].apply(int)

review_by_user = training.groupby('user_int_id').text.sum()
review_by_business = training.groupby('biz_int_id').text.sum()

In [ ]:
#review_by_user.to_csv('review_by_user.csv')
#review_by_business.to_csv('review_by_business.csv')

In [195]:
review = training['text'] 
topic_word_matrix,feature_dict = lda(review,n_topic=10,n_top_words=20)

/Users/mjy/Desktop/Anaconda3/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


perplexity is:  1008.7512013664086
Topic #0:
night bar drink glass bartend cocktail nail hour club le water la date tabl bottl loung patio girl shot vibe
Topic #1:
chicken dish roll sauc soup rice salad beef restaur sushi spici meat flavor meal shrimp steak place pork potato menu
Topic #2:
coffe breakfast cream ice egg tea chocol flavor cake place dessert bacon cooki day sandwich tast fruit brunch cup waffl
Topic #3:
food servic place time restaur order price staff menu server dinner peopl experi star tabl wait minut meal portion everyth
Topic #4:
time vega peopl dog fun place music game friend seat year la parti event line everyon group way thing son
Topic #5:
time servic custom work car day job care staff thank year experi compani busi guy home phone price minut offic
Topic #6:
room hotel time hair area place staff park pool class floor day machin cut locat peopl year bathroom lot way
Topic #7:
pizza beer select place wine time spot sandwich wing salad bread order list crust deliveri

In [196]:
review_by_user = pd.DataFrame(training.groupby('user_int_id').text.sum())  #index: user_int_id, column='text'
review_by_business = pd.DataFrame(training.groupby('biz_int_id').text.sum())

In [198]:
review_by_user['text'].apply(lambda x: topic_probability(x,feature_dict,topic_word_matrix))

user_int_id
0        {0: 1061.014864514111, 1: 1855.3580856209928, ...
1        {0: 13912.773006129635, 1: 85917.59344165497, ...
2        {0: 458.6097052269172, 1: 14655.200141416446, ...
3        {0: 61430.6674076248, 1: 59274.265743235286, 2...
4        {0: 5545.214049928087, 1: 30826.34591631713, 2...
5        {0: 27345.40806521046, 1: 298601.3050848628, 2...
6        {0: 93177.03328936029, 1: 350267.69062919164, ...
7        {0: 5983.918967256919, 1: 61635.27744214379, 2...
8        {0: 27936.012458051984, 1: 52257.9848778678, 2...
9        {0: 367.2193425254425, 1: 1142.4274750745064, ...
10       {0: 13508.134267592694, 1: 156536.6653177368, ...
11       {0: 34496.919664962414, 1: 6678.497880623954, ...
12       {0: 7819.17205801262, 1: 270713.28985019145, 2...
13       {0: 8281.467892442768, 1: 32000.696645465665, ...
14       {0: 3987.897094887616, 1: 50910.77555150958, 2...
15       {0: 7729.692785638037, 1: 111714.55557177155, ...
16       {0: 576.002058264633, 1: 84680.4924

In [200]:
review_by_user['topic_pro_list'] = (review_by_user['text'].
                                    apply(lambda x: topic_probability(x,feature_dict,topic_word_matrix)).
                                    apply(normalize))
review_by_business['topic_pro_list'] = (review_by_business['text'].
                                        apply(lambda x: topic_probability(x,feature_dict,topic_word_matrix)).
                                        apply(normalize))

In [101]:
#calculate the KL between two variables
def KL(a, b):
    a = np.asarray(a, dtype=np.float)
    b = np.asarray(b, dtype=np.float)
    return np.sum(np.where(a != 0, a * np.log(a / b), 0))

values1 = [1.346112,1.337432,1.246655, 0]
values2 = [1.033836,1.082015,1.117323, 0.01]

print(KL(values1, values2))

#calculate the KL similarity between two variables
def KL_sim(a,b):
    a = np.array(list(map(lambda x: 1e-4 if x==0 else x, a)))
    a = np.asarray(a, dtype=np.float)
    b = np.array(list(map(lambda x: 1e-4 if x==0 else x, b)))
    b = np.asarray(b, dtype=np.float)
    KL_ab = np.sum(np.where(a != 0, a * np.log(a/b), 0))
    KL_ba = np.sum(np.where(b != 0, b * np.log(b/a),0))
    return np.exp(-(KL_ab+KL_ba)/2)

KL_sim(values1, values2) #0.9274102245965972   1e-4
#0.002983985448228528    1e-4
#0.0007321799208767189   1e-5
#0.00017971959287665066  1e-6
#4.4115655560183375e-05  1e-7

0.7752796240788413


/Users/mjy/Desktop/Anaconda3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/mjy/Desktop/Anaconda3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """


#### KL Similarity Function          <font color='red'>TODO: Parameters tuning - Is replacing 0 with 1e-4 the best choice?</font>

Why replace 0 with a small number:  np.log(a/b)

In [103]:
import scipy
scipy.stats.entropy(values1,values2)

0.9065084978369279

In [96]:
def KL_sim(a,b):
    KL_ab = scipy.stats.entropy(a,b)
    KL_ba = scipy.stats.entropy(b,a)
    return np.exp(-(KL_ab+KL_ba)/2)

KL_sim(values1, values2)

0.5065062664852481

#### create the businessID_listID dictionary

#### Get User-Topic/Business-Topic Numpy Matrix

In [202]:
user_num = len(review_by_user['topic_pro_list'])
user_topic = np.zeros([user_num, 10])
for i in range(user_num):
    for j in range(0,10):
        user_topic[i][j] = review_by_user['topic_pro_list'][i][j] # row: user; column: topic
        
user_topic.shape

(45770, 10)

In [203]:
business_num = len(review_by_business['topic_pro_list'])
business_topic =  np.zeros([business_num, 10])
for i in range(business_num):
    for j in range(0,10):
        business_topic[i][j]=review_by_business['topic_pro_list'][i][j]  #row; business_id; column:topic

In [204]:
business_topic.shape

(10968, 10)

#### Calculate User-Business Probability Matrix

In [205]:
user_business_pro = np.dot(user_topic,business_topic.T)

In [206]:
user_business_pro.shape # row : user, column = business

(45770, 10968)

In [211]:
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation, cosine

def findksimilarusers(user_id, ratings, metric = 'cosine', k=1):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
    model_knn.fit(ratings)

    distances, indices = model_knn.kneighbors(ratings.iloc[user_id-1, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
    print('{0} most similar users for User {1}:\n'.format(k,user_id))
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;

        else:
            print('{0}: User {1}, with similarity of {2}'.format(i, indices.flatten()[i]+1, similarities.flatten()[i]))
            
    return similarities,indices

In [214]:
import time

start = time.time()
similarities,indices = findksimilarusers(1, pd.DataFrame(user_business_pro), metric='cosine', k =5)
end = time.time()
print('time used:', start - end)

5 most similar users for User 1:

1: User 22361, with similarity of 0.9999969791877369
2: User 11374, with similarity of 0.9999964037324052
3: User 43616, with similarity of 0.9999962657738
4: User 15375, with similarity of 0.9999961354682665
5: User 13922, with similarity of 0.9999961131844802
time used: -114.18974685668945


In [217]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, ratings, metric = 'cosine', k=5):
    prediction=0
    similarities, indices=findksimilarusers(user_id, ratings,metric, k) #similar users based on cosine similarity
    mean_rating = ratings.loc[user_id-1,:].mean() #to adjust for zero based indexing
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0 
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;
        else: 
            ratings_diff = ratings.iloc[indices.flatten()[i],item_id-1]-np.mean(ratings.iloc[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product
    
    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    print('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [220]:
start = time.time()
predict_userbased(3,4,pd.DataFrame(user_business_pro));
end = time.time()
print('time used:', start - end)

5 most similar users for User 3:

1: User 30313, with similarity of 0.9999542862521746
2: User 12527, with similarity of 0.9999519626136075
3: User 4158, with similarity of 0.9999449538228273
4: User 43316, with similarity of 0.9999257201648606
5: User 23083, with similarity of 0.999923507290708

Predicted rating for user 3 -> item 4: 0
time used: -87.60992002487183


#### Calculate User-User Similarity and Business-Business Similarity

In [207]:
user_similarity = np.zeros([user_num, user_num])
for i in range(user_num):
    for j in range(user_num):
        if i== j:
            user_similarity[i][j] = 1
        else:
            user_similarity[i][j] = KL_sim(user_business_pro[i],user_business_pro[j])

KeyboardInterrupt: 

In [ ]:
business_similarity = np.zeros([business_num,business_num])
for i in range(business_num):
    for j in range(business_num):
        if i==j:
            business_similarity[i][j] = 1
        else: 
            business_similarity[i][j] = KL_sim(user_business_pro[:,i], user_business_pro[:,j])

### User-User recommender   
#### <font color='red'>TODO: Parameters tuning - how many top similar users should be chosen? K = 10 </font>

In [124]:
def get_top_k_similar_user(user_id = None,k= 15, biz_id = None):
    #get the similar user_id who have rated the item
    user_rated_item_id = [id_ for id_ in range(num_user) if rating_matrix[id_,biz_id]!=0]
    #find the most similar user
    if len(user_rated_item_id)< k:
        return 'not enough similar users'
    else:
        #[1,4,6,7]
        index_list = np.argsort(user_similarity[user_id])[-k-1:-1]
#    user_ID_list = [user_ID_dic[i] for i in index_list]
    return index_list

top_similar_user_list = get_top_k_similar_user(user_id=0,k = 15)
top_similar_user_list

array([423, 347, 502, 134,  25, 251, 413, 235, 149, 186, 313, 515, 554,
       227, 260])

In [ ]:
def get_top_k_similar_user(user_id = None,k= 15, biz_id = None):
    #get the similar user_id who have rated the item
    user_rated_item_id = [id_ for id_ in range(num_user) if rating_matrix[id_,biz_id]!=0]
    #find the most similar user
    if len(user_rated_item_id)< k:
        return 'not enough similar users'
    else:
        index_list = (pd.DataFrame(user_similarity[user_id]).
                      loc[user_rated_item_id].
                      sort_values(by = 0, ascending = False)[1:k+1].index)
        return index_list

In [ ]:
get_top_k_similar_user(user_id = 10, k= 15, biz_id = 5)

In [ ]:
# user dictionary (raw ID) - (assigned int ID)
user_raw_int_dic = {value:key for key, value in user_ID_dic.items()}

In [78]:
user_mean_rating_dic = training.groupby('user_id').stars.mean().to_dict()
user_mean_rating_dic = {user_raw_int_dic[key]:_ for key, _ in user_mean_rating_dic.items()}
# user_mean_rating_dic {int_ID: mean_rating}

In [132]:
user_mean_rating_dic = training.groupby('user_int_id').stars.mean().to_dict()

<font color='red'>TODO: Creating a user_biz rating matrix (with int ID) </font>

In [125]:
training[:5]

,Unnamed: 0,business_id,user_id,stars,text,user_int_id,biz_int_id
0,0,ujmEBvifdJM6h6RLv4wQIg,hG7b0MtEbXx5QzbzE6C_VA,1.0,bill servic crook pill pill cent hospit er cost,0,0
1,1,NZnhc2sEQy3RmzKTZnqtwQ,yXQM5uF2jS6es16SJzNHfg,5.0,rock salon stranger chain servic flawless leve...,1,1
2,2,WTqjgwHlXbSFevF32_DJVw,n6-Gk65cPZL6Uz8qRm3NYw,5.0,offic j phillipp dentist dental assist procedu...,2,2
3,3,ikCg8xy5JIg_NGPx-MSIDA,dacAIZ6fTM6mqwW5uxkskg,5.0,steak sandwich caesar salad amount leaf drink ...,3,3
4,4,b1b1eb3uo-w561D0ZfCEiQ,ssoyf2_x0EQMed6fgHeMyQ,1.0,today session session enjoy male client fine r...,4,4


In [137]:
from scipy import sparse
import numpy as np
 
row = [1,2, 3, 3, 2]
col = [1,3, 4, 2, 3]
data = [3,5, 2, 3, 10]
c = sparse.csr_matrix((data, (row, col)), shape=(5, 5))

print(c.toarray())

[[ 0  0  0  0  0]
 [ 0  3  0  0  0]
 [ 0  0  0 15  0]
 [ 0  0  3  0  2]
 [ 0  0  0  0  0]]



In [144]:
#Generate Rating Sparse Matrix
user_row = training['user_int_id'].values
biz_column = training['biz_int_id'].values

user_num = len(user_raw_int_id_dic)
biz_num = len(biz_raw_int_id_dic)

rating_data = training['stars'].values
rating_matrix = sparse.csr_matrix((rating_data, (user_row, biz_column)), shape=(user_num, biz_num))

In [173]:
def rating_prediction(user_id = 1, biz_id = 10, top_k = 15):
    user_i_mean_rating = user_mean_rating_dic[user_id]
    top_similar_user_list = get_top_k_similar_user(user_id ,k = top_k)
    #list to store user info: mean rating, rating given biz_id, similarity
    u_info = []
   
    for similar_user in top_similar_user_list:
        mean_rating = user_mean_rating_dic[similar_user]        
        rating_u_i = rating_matrix[similar_user, biz_id]
        similarity = user_similarity[user_id][similar_user]
        u_info.append([mean_rating, rating_u_i, similarity])
    
    similar_user_rating = np.sum([(u_info[i][1] - u_info[i][0])*u_info[i][2] for i in range(top_k)])
    sum_of_similarity = np.sum([u_info[i][2] for i in range(top_k)])
    predicted_rating = user_i_mean_rating+ similar_user_rating/sum_of_similarity
    return predicted_rating
    
rating_prediction(user_id = 5, biz_id = 2, top_k = 15)

0.5283383351520485

In [168]:
user_rating_prediction_list = []
for i in range(300):
    user_rating_prediction_list.append(rating_prediction(user_id = 5, biz_id = i, top_k = 15))
user_rating_prediction_list

[0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.8834872202102697,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5283383351520485,
 0.5954787175746898,
 0.5283383351520485,
 0.5283383351520485,
 0.725473742408274,
 0.5283383351520485,
 0.595948925714203,
 0.5283383351520485,
 0.5283383351520485,
 0.8455041844667002,
 0.5283383351520485,
 0.5944138862293311,
 0.5283383351520485,
 0.5283383351520485,
 0.528338335152

### Business-Business Recommender

In [ ]:
#1. first get the top 10 users

def get_top_k_similar_business(user_id,k):
    #get the index of most similar user
    index_list = np.argsort(business_similarity[1])[-k-1:-1]
    biz_ID_list = [user_ID_dic[i] for i in index_list]
    return biz_ID_list